In [27]:
# Or you can use a .env file
import os
from dotenv import load_dotenv
load_dotenv(dotenv_path=".env", override=True)  # .env is in the current Module_1 folder

True

In [28]:
import uuid
thread_id = uuid.uuid4()

In [29]:
from langsmith import traceable
from groq import Groq
from typing import List
import nest_asyncio
from utils import get_vector_db_retriever
import os

groq_key = os.getenv("GROQ_API_KEY")
groq_client = Groq(api_key=groq_key)
nest_asyncio.apply()
retriever = get_vector_db_retriever()

@traceable(run_type="chain")
def retrieve_documents(question: str):
    return retriever.invoke(question)

@traceable(run_type="chain")
def generate_response(question: str, documents):
    formatted_docs = "\n\n".join(doc.page_content for doc in documents)
    rag_system_prompt = """You are an assistant for question-answering tasks. 
    Use the following pieces of retrieved context to answer the latest question in the conversation. 
    If you don't know the answer, just say that you don't know. 
    Use three sentences maximum and keep the answer concise.
    """
    messages = [
        {
            "role": "system",
            "content": rag_system_prompt
        },
        {
            "role": "user",
            "content": f"Context: {formatted_docs} \n\n Question: {question}"
        }
    ]
    return call_groq(messages)

@traceable(run_type="llm")
def call_groq(
    messages: List[dict], model: str = "moonshotai/kimi-k2-instruct-0905", temperature: float = 0.0
) -> str:
    return groq_client.chat.completions.create(
        model=model,
        messages=messages,
        temperature=temperature,
    )

@traceable(run_type="chain")
def langsmith_rag(question: str):
    documents = retrieve_documents(question)
    response = generate_response(question, documents)
    return response.choices[0].message.content


In [30]:
question = "How do I add metadata to a Trace?"
ai_answer = langsmith_rag(question)
print(ai_answer)

LangSmith lets you attach metadata as a dictionary of key-value pairs when you log a trace. In Python you can supply a `metadata` argument (and `tags`) directly in the LangSmith tracing call. Both metadata and tags are then stored with the trace and can be used later to filter or query traces in the UI or SDK.


In [31]:
question = "How can I add tags to a Trace?"
ai_answer = langsmith_rag(question)
print(ai_answer)

You can include tags when you emit the trace to LangSmith (for example, with the OpenAI wrapper's request or via the LangSmith SDK), and LangSmith accepts a list/array of strings for the `tags` field. To set these tags easily, wrap your OpenAI calls with the LangSmith utility `wrap_openAI()` in Python (or the equivalent wrappers in JS/TS) and pass tags as a parameter.
